# Part 8 - Federated Learning on MNIST using a CNN

We show here how  simple it is to switch to Pytorch models to syft models so as to perform Federated Learning, by using directly the canonical example of training a CNN on MNIST which provided by PyTorch [here](https://github.com/pytorch/examples/blob/master/mnist/main.py). We only changed the code to run in a notebook and we mention all the changed made which enable Federated Learning.

**Roadmap of future improvements:**

 _See the #TODO for details about the following issues:_
 - support **momentum args in optim SGD** #1896
 - **use `FederatedDataset` & `FederatedDataLoader` @jlebensold**
 - support .to(device) hooking #1893
 - silence the Warning in log_softmax -> needs a hook_kwargs


Authors:
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

### Imports and model specifications

First we make the official imports

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

And than those specific to PySyft. In particular we define remote workers alice and bob.

In [2]:
import syft as sy
import torch
import torch.nn.functional as F
import json
import random
hook = sy.TorchHook(torch)
me = hook.local_worker
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)
me.is_client_worker = False

compute_nodes = [bob, alice]

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

We define the setting of the learning task

In [9]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Data loading
We first load the data...

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

... and then send the training dataset to the workers, alice and bob.

In [5]:
#TODO leverage FederatedDataset and FederatedDataloader
train_distributed_dataset = []

n_batch = 937

for batch_idx, (data,target) in enumerate(train_loader):
    if batch_idx % 20 == 0:
        print(round(100*batch_idx/n_batch), '%')
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    train_distributed_dataset.append((data, target))
    if batch_idx > 100: #TODO rm this limit which is just for dev to have less data to load and train
        break
        
print('Done !')

0 %
2 %
4 %
6 %
9 %
11 %
Done !


### CNN specification
Here we use exactly the same CNN as in the official example.

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1) #TODO: shouldn't raise warning

### Define the train and test functions

In [7]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_distributed_dataset): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        #TODO data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            #TODO should be using with kwargs: output.argmax(dim=1, keepdim=True)
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


### Launch the training !

In [8]:
model = Net() #TODO .to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) #TODO , momentum=args.momentum

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")


/Users/ryffel/Documents/Code/PySyft/syft/frameworks/torch/tensors/interpreters/native.py:185: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  response = eval(cmd)(*args)


Train Epoch: 1 [0/60032 (0%)]	Loss: 2.281877
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.230119
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.170815
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.162510
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.070279
Train Epoch: 1 [3200/60032 (5%)]	Loss: 1.946969
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.825168
Train Epoch: 1 [4480/60032 (7%)]	Loss: 1.627585
Train Epoch: 1 [5120/60032 (9%)]	Loss: 1.354165
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.098111
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.904220

Test set: Average loss: 0.0010, Accuracy: 7949/10000 (79%)

Train Epoch: 2 [0/60032 (0%)]	Loss: 0.798303
Train Epoch: 2 [640/60032 (1%)]	Loss: 0.786481
Train Epoch: 2 [1280/60032 (2%)]	Loss: 0.631002
Train Epoch: 2 [1920/60032 (3%)]	Loss: 0.805443
Train Epoch: 2 [2560/60032 (4%)]	Loss: 0.681612
Train Epoch: 2 [3200/60032 (5%)]	Loss: 0.456450
Train Epoch: 2 [3840/60032 (6%)]	Loss: 0.593717
Train Epoch: 2 [4480/60032 (7%)]	Loss: 0.696700
Train Epoch: 2 [5120/60032 (9%)]	